In [62]:
from pathlib import Path
import tifffile
import napari
from apoc import ObjectSegmenter, PixelClassifier
from skimage import measure
import tqdm
import pandas as pd
import os

In [39]:
# Define the directory containing your files
directory_path = Path("./data/before_sonication/")

# Initialize an empty list to store the images
images = []

# Iterate through the files in the directory
for file_path in directory_path.glob('*.TIF'):
    
    # Append the file to the corresponding group
    images.append(str(file_path))

In [40]:
# This is just a check, it prints the image filepaths contained in images
for filepath in images:
    print(filepath)

data\before_sonication\plate 1_Top Slide_TD_p00_0_A01f00d1.TIF
data\before_sonication\plate 2_Top Slide_TD_p00_0_A02f00d1.TIF
data\before_sonication\plate 3_Top Slide_TD_p00_0_A03f00d1.TIF


In [21]:
# Load one RGB image and extract the green channel for APOC
rgb_img = tifffile.imread(images[5], is_ome = False)

# Extract green channel [1]
img = rgb_img[:,:,1]

# Visualize greyscale green channel in napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img, name="Biofilm")

<Image layer 'Biofilm' at 0x21a09bb7fd0>

In [22]:
# Apply pretrained object segmenter from APOC
object_segmenter = ObjectSegmenter(opencl_filename="./pretrained_classifiers/ObjectSegmenter.cl")
obj_seg_result = object_segmenter.predict(image=img)
viewer.add_labels(obj_seg_result, name="Biofilm_objects")

<Labels layer 'Biofilm_objects' at 0x21a092eab20>

In [26]:
# Apply pretrained semantic segmenter from APOC
semantic_segmenter = PixelClassifier(opencl_filename="./pretrained_classifiers/PixelClassifier.cl")
sem_seg_result = semantic_segmenter.predict(image=img)
viewer.add_labels(sem_seg_result, name="Biofilm_layer")

<Labels layer 'Biofilm_layer' at 0x21a082a2af0>

In [32]:
# Calculate the area covered by the biofilm
props = measure.regionprops(sem_seg_result) # Calculate the background and biofilm area

# Extract area values
area_values = [prop.area for prop in props]

print(f"Background: {area_values[0]} pixels, Biofilm: {area_values[1]} pixels")

Background: 11975417.0 pixels, Biofilm: 547135.0 pixels


In [56]:
# Create an empty dictionary to store filename:biofilm_area pairs and later convert it into a pandas df
biofilm_areas = {}

# Initialize Napari viewer instance
viewer = napari.Viewer(ndisplay=2)

for filepath in images:
    # Get the filename without the extension
    filename = Path(filepath).stem
    
    # Load one RGB image and extract the green channel for APOC
    rgb_img = tifffile.imread(filepath, is_ome = False)

    # Extract green channel [1]
    img = rgb_img[:,:,1]

    # Visualize greyscale green channel in napari
    viewer.add_image(img, name=f"Biofilm_{filename}")
    
    # Apply pretrained object segmenter from APOC
    object_segmenter = ObjectSegmenter(opencl_filename="./pretrained_classifiers/ObjectSegmenter.cl")
    obj_seg_result = object_segmenter.predict(image=img)
    viewer.add_labels(obj_seg_result, name=f"Biofilm_objects_{filename}")
    
    # Apply pretrained semantic segmenter from APOC
    semantic_segmenter = PixelClassifier(opencl_filename="./pretrained_classifiers/PixelClassifier.cl")
    sem_seg_result = semantic_segmenter.predict(image=img)
    viewer.add_labels(sem_seg_result, name=f"Biofilm_layer_{filename}")
    
    # Calculate the area covered by the biofilm
    props = measure.regionprops(sem_seg_result) # Calculate the background and biofilm area
    
    # Extract area values
    area_values = [prop.area for prop in props]
    
    # Testing purposes
    print(f"Filename: {filename} Background: {area_values[0]} pixels, Biofilm: {area_values[1]} pixels")
    
    # Add new filename:biofilm_area pair to biofilm_areas
    biofilm_areas[filename] = area_values[1]


Filename: plate 1_Top Slide_TD_p00_0_A01f00d1 Background: 8013722.0 pixels, Biofilm: 336742.0 pixels
Filename: plate 2_Top Slide_TD_p00_0_A02f00d1 Background: 12063260.0 pixels, Biofilm: 459292.0 pixels
Filename: plate 3_Top Slide_TD_p00_0_A03f00d1 Background: 7859379.0 pixels, Biofilm: 491085.0 pixels


{'plate 1_Top Slide_TD_p00_0_A01f00d1': 336742.0,
 'plate 2_Top Slide_TD_p00_0_A02f00d1': 459292.0,
 'plate 3_Top Slide_TD_p00_0_A03f00d1': 491085.0}

In [63]:
# Convert dictionary to pandas DataFrame
df = pd.DataFrame(list(biofilm_areas.items()), columns=['filename', 'biofilm_area'])

# Define the output folder path
output_folder = './output/'

# Check if the output folder exists, create it if not
try:
    os.makedirs(output_folder)
    print(f"Output folder created: {output_folder}")
except FileExistsError:
    print(f"Output folder already exists: {output_folder}")

# Save the DataFrame to a CSV file in the output folder
df.to_csv(os.path.join(output_folder, 'biofilm_areas.csv'), index=False)

# Display the resulting dataframe
df

Output folder already exists: ./output/


,filename,biofilm_area
0,plate 1_Top Slide_TD_p00_0_A01f00d1,336742.0
1,plate 2_Top Slide_TD_p00_0_A02f00d1,459292.0
2,plate 3_Top Slide_TD_p00_0_A03f00d1,491085.0
